#**Hotel Reviews NLP Project**

####Names: MacKenna Landry, Bryanna Boone

###**Data Pre-Processing**

https://www.kaggle.com/datasets/juhibhojani/hotel-reviews

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd

hotel_data = pd.read_csv("hotel_reviews.csv")
hotel_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7001 entries, 0 to 7000
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Index              7001 non-null   int64  
 1   Name               7001 non-null   object 
 2   Area               7001 non-null   object 
 3   Review_Date        7001 non-null   object 
 4   Rating_attribute   7001 non-null   object 
 5   Rating(Out of 10)  7001 non-null   float64
 6   Review_Text        6994 non-null   object 
dtypes: float64(1), int64(1), object(5)
memory usage: 383.0+ KB


In [2]:
hotel_data.head()

,Index,Name,Area,Review_Date,Rating_attribute,Rating(Out of 10),Review_Text
0,0,Hotel The Pearl,"Paharganj, New Delhi",Jul-23,Best budget friendly hotel,9.0,Hotel the pearl is perfect place to stay in De...
1,1,Hotel The Pearl,"Paharganj, New Delhi",Aug-23,Amazing place,9.0,Location of the hotel is perfect. The hotel is...
2,2,Hotel The Pearl,"Paharganj, New Delhi",Aug-23,Overall good stay. Economic.,9.0,"Location, Indian food."
3,3,Hotel The Pearl,"Paharganj, New Delhi",Aug-23,Lovely,9.0,The location and the hotel itself is great. Ne...
4,4,Hotel The Pearl,"Paharganj, New Delhi",Aug-23,Great hotel Great staff and great staying,9.0,Friendly and smiling staffs.. The reception st...


In [3]:
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def clean_and_tokenize_text(text):
    if pd.isnull(text):
        return ''

    # Tokenization using regular expression
    tokens = re.findall(r'\b\w+\b', text.lower())

    # Stopword removal and lemmatization
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    clean_tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return ' '.join(clean_tokens)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [4]:
hotel_data['Rating_attribute'] = hotel_data['Rating_attribute'].apply(clean_and_tokenize_text)
hotel_data['Review_Text'] = hotel_data['Review_Text'].apply(clean_and_tokenize_text)

In [5]:
hotel_data.head()

,Index,Name,Area,Review_Date,Rating_attribute,Rating(Out of 10),Review_Text
0,0,Hotel The Pearl,"Paharganj, New Delhi",Jul-23,best budget friendly hotel,9.0,hotel pearl perfect place stay delhi paharganj...
1,1,Hotel The Pearl,"Paharganj, New Delhi",Aug-23,amazing place,9.0,location hotel perfect hotel peaceful nice sta...
2,2,Hotel The Pearl,"Paharganj, New Delhi",Aug-23,overall good stay economic,9.0,location indian food
3,3,Hotel The Pearl,"Paharganj, New Delhi",Aug-23,lovely,9.0,location hotel great next time stay nice room ...
4,4,Hotel The Pearl,"Paharganj, New Delhi",Aug-23,great hotel great staff great staying,9.0,friendly smiling staff reception staff excelle...


###**Sentiment Analysis using VADER**

In [6]:
pip install vaderSentiment

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.2 MB/s eta 0:00:00


In [7]:
# import SentimentIntensityAnalyzer class
# from vaderSentiment.vaderSentiment module.
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# function to print sentiments
# of the sentence.
def sentiment_scores(sentence):

    # Create a SentimentIntensityAnalyzer object.
    sid_obj = SentimentIntensityAnalyzer()

    # polarity_scores method of SentimentIntensityAnalyzer
    # object gives a sentiment dictionary.
    # which contains pos, neg, neu, and compound scores.
    sentiment_dict = sid_obj.polarity_scores(sentence)
    return sentiment_dict['compound']

def get_sentiment_label(score):
    if score >= 0.05:
        return 'Positive'
    elif score <= -0.05:
        return 'Negative'
    else:
        return 'Neutral'


In [8]:
#Calculate sentiment score for each review
hotel_data['Sentiment_value'] = hotel_data['Review_Text'].apply(sentiment_scores)


In [9]:
hotel_data.head()

,Index,Name,Area,Review_Date,Rating_attribute,Rating(Out of 10),Review_Text,Sentiment_value
0,0,Hotel The Pearl,"Paharganj, New Delhi",Jul-23,best budget friendly hotel,9.0,hotel pearl perfect place stay delhi paharganj...,0.9633
1,1,Hotel The Pearl,"Paharganj, New Delhi",Aug-23,amazing place,9.0,location hotel perfect hotel peaceful nice sta...,0.9545
2,2,Hotel The Pearl,"Paharganj, New Delhi",Aug-23,overall good stay economic,9.0,location indian food,0.0000
3,3,Hotel The Pearl,"Paharganj, New Delhi",Aug-23,lovely,9.0,location hotel great next time stay nice room ...,0.9423
4,4,Hotel The Pearl,"Paharganj, New Delhi",Aug-23,great hotel great staff great staying,9.0,friendly smiling staff reception staff excelle...,0.9750


In [10]:

# Group the DataFrame by hotel name and calculate the average sentiment score for each hotel
hotel_sentiment = hotel_data.groupby('Name').agg({'Sentiment_value': 'mean', 'Review_Text': 'first'}).reset_index()

# Sort the hotels based on their average sentiment scores to rank them
ranked_hotels = hotel_sentiment.sort_values(by='Sentiment_value', ascending=False)


In [11]:
sentiment_df = pd.DataFrame({
    'Hotel Name': ranked_hotels['Name'],
    'Sentiment Label': ranked_hotels['Sentiment_value'].apply(get_sentiment_label),
    'Sentiment Score': ranked_hotels['Sentiment_value'],
    'Review Text': ranked_hotels['Review_Text']
})

In [12]:
num_rows = sentiment_df.shape[0]
print("Number of Hotels Reviewed: " + str(num_rows))
print()
print("Examples of the highest ranked hotels based on their sentiment scores:")
sentiment_df[:5]
print("Examples of the lowest ranked hotels based on their sentiment scores:")
sentiment_df[565:]


Number of Hotels Reviewed: 570

Examples of the highest ranked hotels based on their sentiment scores:


,Hotel Name,Sentiment Label,Sentiment Score,Review Text
7,Abuzz Oxfordcaps ShahpurJaat Delhi,Positive,0.9521,property shahpurjat girl property serf veg foo...
192,Hotel City Plaza,Positive,0.9359,lovely stay room comfy clean service every day...
158,Homlee Saket Basement Flat with Bathroom at Gr...,Positive,0.9313,place convenient location wise perfect planing...
477,Roomshala 120 Hotel Crown Near Signature Bridg...,Positive,0.9300,great experience roomshala hotel crown great s...
140,HOTEL CITY INN,Positive,0.9246,pocket friendly hotel good staff service clean...


Examples of the lowest ranked hotels based on their sentiment scores:


,Hotel Name,Sentiment Label,Sentiment Score,Review Text
56,Collection O The Pinaki Residency Near Dlf Ave...,Negative,-0.340125,dirty bed sheet bad odour room
569,noratan palace,Negative,-0.361533,bad condition toilet
534,Townhouse 1251 The Royce By Dsr,Negative,-0.440400,hotel operational quite time still listed even...
34,Bewafa Palace,Negative,-0.762500,let check left road midnight worst experience ...
524,The New Anand Near Karol Bagh Metro Station,Negative,-0.972900,booking com representative respond call denyin...


#Sentence Embeddings to Find Similar Hotels to Query

In [13]:
# partition sentiment df to only include positive reviews to recommend to users
recommend_df = sentiment_df[:450]
recommend_df

,Hotel Name,Sentiment Label,Sentiment Score,Review Text
7,Abuzz Oxfordcaps ShahpurJaat Delhi,Positive,0.952100,property shahpurjat girl property serf veg foo...
192,Hotel City Plaza,Positive,0.935900,lovely stay room comfy clean service every day...
158,Homlee Saket Basement Flat with Bathroom at Gr...,Positive,0.931300,place convenient location wise perfect planing...
477,Roomshala 120 Hotel Crown Near Signature Bridg...,Positive,0.930000,great experience roomshala hotel crown great s...
140,HOTEL CITY INN,Positive,0.924600,pocket friendly hotel good staff service clean...
...,...,...,...,...
54,Collection O Radisson Blue Hotel Nearby Khajur...,Positive,0.110100,red light area
241,Hotel Jai Deluxe- Paharganj New Delhi Railway ...,Positive,0.109860,staff nice helpful
391,OYO Flagship 75287 Pubg Residency,Positive,0.106217,location good near metro station
242,Hotel Jashn Inn near Apollo Hospital,Positive,0.101575,breakfast excellent location also excellent


In [14]:
reviews = recommend_df["Review Text"]

reviews[:5]

7      property shahpurjat girl property serf veg foo...
192    lovely stay room comfy clean service every day...
158    place convenient location wise perfect planing...
477    great experience roomshala hotel crown great s...
140    pocket friendly hotel good staff service clean...
Name: Review Text, dtype: object

In [15]:
#user input
query = input()
query = str(query)

good hotel, very nice staff, good views, convenient location


In [16]:
print(query)

good hotel, very nice staff, good views, convenient location


Word2Vec

In [17]:
import gensim.downloader as api

# Load the pre-trained Word2Vec model

# Download a pre-trained word2vec (trained on Google News data)
w2v_model = api.load("word2vec-google-news-300")

# Compute similarity between words
def compute_similarity(model, word1, word2):
    try:
        return model.similarity(word1, word2)
    except KeyError:
        return 0  # return 0 if either of the words is not in the vocabulary

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [18]:
# Function to extract sentence vector from word vectors by averaging word embeddings
from scipy.spatial.distance import cosine
import numpy as np

def extract_sentence_vector(sentence):
    words = sentence.split()
    word_vectors = [w2v_model[word] for word in words if word in w2v_model]
    if not word_vectors:
        return None  # Return None if no word vectors are found
    sentence_vector = np.mean(word_vectors, axis=0)
    return sentence_vector

def similarity (x1, x2):
  # similarity is the opposite of distance
  return 1 - cosine(x1, x2)

In [19]:
# Example of extracting sentence vector using mean of word vectors

# change hotel names to list so that we can get hotel recs by index nums
hotel_list = recommend_df["Hotel Name"].tolist()

#process dataset to get a list of sentencese (the reviews)
review_list = reviews.tolist()#lowercase and append to list
print(review_list[:5])
#extract sentence vectors and store in list
vector_list = []
for review in review_list:
  sentence_vector = extract_sentence_vector(review)
  vector_list.append(sentence_vector)

['property shahpurjat girl property serf veg food taste really amazing washroom dorm really clean hygenic bed really cozy clean privacy curtain 24hrs ac 3time meal included 170 rupee nice cabin room dorm accommodate 1person looking privacy long stay overall priyanka warden property soo sweet decent book property without second thought totally worth', 'lovely stay room comfy clean service every day receptionist team friendly professional breakfast service nice fast would highly recommend hotel', 'place convenient location wise perfect planing go mehrauli pretty often place stay loved hospitality', 'great experience roomshala hotel crown great stay good location property clean hygienic', 'pocket friendly hotel good staff service clean room near metro station hence help easy commute various place']


In [20]:
query = query.lower()
#print(query)
sentence_vector2 = extract_sentence_vector(query)

#compute all similarities
similarity_list = []
new_hotel = []
i = 0
for vector in vector_list:
  if vector is not None:
    sim = similarity(vector, sentence_vector2)
    similarity_list.append(sim)
    new_hotel.append(hotel_list[i])

  i +=1

print("Some similarity scores", similarity_list[:5])

#hotel_list and similarity_list are parallel - create dictionary
hotel_dict = {}
for i, hotel in enumerate(new_hotel):
  hotel_dict[hotel] = similarity_list[i]


sorted_hotel = dict(sorted(hotel_dict.items(), key=lambda item: item[1]))
#print(len(sorted_hotel))

top_hotel = list(sorted_hotel.keys())[444:]
reviews =  list(sorted_hotel.values())[444:]
num = 1
for i in range(4, -1, -1):
  print(f"{num}. {top_hotel[i]}: {reviews[i]}")
  num+=1

Some similarity scores [0.6021687388420105, 0.6660736203193665, 0.7880666255950928, 0.7228183746337891, 0.6117616891860962]
1. Hotel Stay Villa - Couple Friendly - 10 mints from Chattarpur metro station: 0.8628001809120178
2. Hotel Roma Deluxe: 0.860352635383606
3. Hotel Amar Inn- Lajpat Nagar Central Market: 0.8572825193405151
4. Hotel pathway near Delhi airport: 0.8522619605064392
5. Hotel New City Lite Near IGI Airport Delhi: 0.8372957706451416


Using BERT

In [ ]:
!pip install sentence_transformers

In [22]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [23]:
# change hotel names to list so that we can get hotel recs by index nums
hotel_list2 = recommend_df["Hotel Name"].to_list()

# Load a pre-trained BERT model
model = SentenceTransformer('bert-base-uncased')

#process dataset to get a list of sentencese (the reviews)
review_list2 = []
for review in review_list:
  review_list2.append(review) #lowercase and append to list

print(review_list2[:5])

sentence_embeddings = model.encode(review_list2)

query_embeddings = model.encode([query])

# Compute similarity between the first two sentences
similarity2 = []
for embed in sentence_embeddings:
  similarity_score = similarity(embed,query_embeddings[0])
  similarity2.append(similarity_score)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

['property shahpurjat girl property serf veg food taste really amazing washroom dorm really clean hygenic bed really cozy clean privacy curtain 24hrs ac 3time meal included 170 rupee nice cabin room dorm accommodate 1person looking privacy long stay overall priyanka warden property soo sweet decent book property without second thought totally worth', 'lovely stay room comfy clean service every day receptionist team friendly professional breakfast service nice fast would highly recommend hotel', 'place convenient location wise perfect planing go mehrauli pretty often place stay loved hospitality', 'great experience roomshala hotel crown great stay good location property clean hygienic', 'pocket friendly hotel good staff service clean room near metro station hence help easy commute various place']


In [24]:
#hotel_list and similarity_list are parallel - create dictionary
hotel_dict2 = {}
for i,hotel in enumerate(hotel_list2):
  hotel_dict2[hotel] = similarity2[i]


sorted_hotel2 = dict(sorted(hotel_dict2.items(), key=lambda item: item[1]))

top_hotel2 = list(sorted_hotel2.keys())[444:]
reviews2 =  list(sorted_hotel2.values())[444:]
#printing out top 5 most similar hotels
print("Best hotels for you:")
num = 1
for i in range(4, -1, -1):
  print(f"{num}. {top_hotel2[i]}: {reviews2[i]}")
  num+=1

Best hotels for you:
1. Townhouse 511 Prakash continental Near Netaji Subhash Place Metro Station: 0.8283188939094543
2. OYO Royal Galaxy: 0.8259118795394897
3. Hotel Admire Gold - Pachim Vihar Delhi: 0.8170830607414246
4. Hotel Anand International: 0.8095898628234863
5. Hotel Airport JMD Divine 15 MINTUE IGI AIRPORT DELHI: 0.8048772215843201
